## LLM chat completions

In [1]:
from openai import OpenAI
import json
from rich import print

import dotenv
dotenv.load_dotenv()

True

In [2]:
client = OpenAI()

In [3]:
messages = [{"role": "user", "content": "What's the weather like in London?"}]
response = client.chat.completions.create(
    model="gpt-4o",
    messages=messages,
)

In [4]:
print(response)

ChatCompletion(
    id='chatcmpl-9VpIJALaA6vzFVBja2GoRVXzXiRox',
    choices=[
        Choice(
            finish_reason='stop',
            index=0,
            logprobs=None,
            message=ChatCompletionMessage(
                content="I'm unable to provide real-time information, including the current weather. To find out 
the weather in London right now, you can check a reliable weather website, use a weather app, or ask a virtual 
assistant equipped with real-time data capabilities.",
                role='assistant',
                function_call=None,
                tool_calls=None
            )
        )
    ],
    created=1717372371,
    model='gpt-4o-2024-05-13',
    object='chat.completion',
    system_fingerprint='fp_319be4768e',
    usage=CompletionUsage(completion_tokens=48, prompt_tokens=14, total_tokens=62)
)

In [5]:
def get_current_weather(location):
    """Get the current weather in a given city"""
    if "london" in location.lower():
        return json.dumps({"temperature": "20 C"})
    elif "san francisco" in location.lower():
        return json.dumps({"temperature": "15 C"})
    elif "paris" in location.lower():
        return json.dumps({"temperature": "22 C"})
    else:
        return json.dumps({"temperature": "unknown"})



messages = [{"role": "user", "content": "What's the weather like in London?"}]
tools = [
    {
        "type": "function",
        "function": {
            "name": "get_current_weather",
            "description": "Get the current weather in a given location",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        "description": "The city and state, e.g. San Francisco",
                    },
                },
                "required": ["location"],
            },
        },
    }
]

In [6]:
response = client.chat.completions.create(
    model="gpt-4o",
    messages=messages,
    tools=tools,
)
print(response)

ChatCompletion(
    id='chatcmpl-9VpILp6Z59Q0sIjRGAk14iktaUie6',
    choices=[
        Choice(
            finish_reason='tool_calls',
            index=0,
            logprobs=None,
            message=ChatCompletionMessage(
                content=None,
                role='assistant',
                function_call=None,
                tool_calls=[
                    ChatCompletionMessageToolCall(
                        id='call_KtSiNep0SfYtrlZGZ5v6umgT',
                        function=Function(arguments='{"location":"London"}', name='get_current_weather'),
                        type='function'
                    )
                ]
            )
        )
    ],
    created=1717372373,
    model='gpt-4o-2024-05-13',
    object='chat.completion',
    system_fingerprint='fp_319be4768e',
    usage=CompletionUsage(completion_tokens=15, prompt_tokens=66, total_tokens=81)
)

## Agents & Tool use

In [7]:
from agents.llms import OpenAIChatCompletion

In [8]:
from agents.tools import get_current_weather
from agents.tool_executor import need_tool_use

In [14]:
llm = OpenAIChatCompletion()
llm.bind_tools([get_current_weather])

In [11]:
messages = [
    {"role": "user", "content": "how is the weather in London today?"}
]

output = llm.chat_completion(messages)

In [12]:
print(output)

ChatCompletion(
    id='chatcmpl-9VpIMyA2V34AZjastE7g98gNzCrSc',
    choices=[
        Choice(
            finish_reason='tool_calls',
            index=0,
            logprobs=None,
            message=ChatCompletionMessage(
                content=None,
                role='assistant',
                function_call=None,
                tool_calls=[
                    ChatCompletionMessageToolCall(
                        id='call_jqvvKaUmVUMl1WLej9AfGLpp',
                        function=Function(arguments='{"city":"London"}', name='get_current_weather'),
                        type='function'
                    )
                ]
            )
        )
    ],
    created=1717372374,
    model='gpt-4o-2024-05-13',
    object='chat.completion',
    system_fingerprint='fp_319be4768e',
    usage=CompletionUsage(completion_tokens=15, prompt_tokens=167, total_tokens=182)
)

In [13]:
if need_tool_use(output):
    print("Using weather tool")
    tool_results = llm.run_tools(output)
    tool_results[0]["role"] = "assistant"

    updated_messages = messages + tool_results
    updated_messages = updated_messages + [
        {"role": "user", "content": "Think step by step and answer my question based on the above context."}
    ]
    output = llm.chat_completion(updated_messages)

print(output.choices[0].message.content)

Using weather tool

To determine the weather in London today based on the provided data, let's go through it step-by-step:

1. **Temperature**:
    - Current temperature is 21°C (70°F).
    - Feels like temperature is also 21°C (70°F).

2. **Weather Condition**:
    - The weather is described as "Clear".

3. **Wind**:
    - Wind direction is North-Northwest (NNW) at a speed of 6 km/h (4 mph).

4. **Precipitation**:
    - No precipitation is reported (0.0 mm or 0.0 inches).

5. **Humidity**:
    - The humidity level is at 43%.

6. **Pressure**:
    - The atmospheric pressure is 1024 hPa (30 inches).

7. **Visibility**:
    - Visibility is 10 km (6 miles).

8. **UV Index**:
    - UV Index is 5, which is moderate.

Given this data, the weather in London today is clear with a temperature of around 21°C (70°F), light NNW wind, and
no precipitation.